<a href="https://colab.research.google.com/github/dk-wei/customer-driven-ds/blob/main/Pandas_Group_%E7%94%A8%E6%B3%95%E8%AF%A6%E8%A7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [185]:
import pandas as pd
import numpy as np

In [186]:
company = ["A","B","C"]

data = pd.DataFrame({"company":[company[x] for x in np.random.randint(0,len(company),10)],
                   "salary":np.random.randint(5,50,10),
                   "age":np.random.randint(15,50,10)})

In [187]:
data.head()

,company,salary,age
0,A,21,15
1,B,35,20
2,A,14,39
3,C,31,38
4,B,29,37


In [188]:
group = data.groupby("company")

In [189]:
group

groupby很简单，直接存入了内存成为了一个对象

要想看到groupby的话可以使用`list`, 或者用`apply(list)`

In [190]:
list(group)

[('A',   company  salary  age
  0       A      21   15
  2       A      14   39
  5       A      13   30), ('B',   company  salary  age
  1       B      35   20
  4       B      29   37
  7       B      19   45
  8       B       7   46
  9       B      40   28), ('C',   company  salary  age
  3       C      31   38
  6       C      10   24)]

In [191]:
list(group)[0][0]

'A'

转换成列表的形式后，可以看到，列表由三个元组组成，每个元组中，第一个元素是组别（这里是按照company进行分组，所以最后分为了A,B,C），第二个元素的是对应组别下的DataFrame，整个过程可以图解如下：

![](https://pic2.zhimg.com/80/v2-c619d636a34458a51b375b0ad2cbf7c5_1440w.jpg)




# 最最重要的底层概念就是：

  groupby的过程就是将原有的DataFrame按照groupby的字段（这里是company），划分为若干个分组的**小DataFrame**，被分为多少个组就有多少个分组DataFrame。**所以说，在groupby之后的一系列操作（如agg、apply等），均是基于小DataFrame的操作**。可以通过groupby后面选取某个column，选取小Dataframe，进行操作。
  
  
理解了这点，也就基本摸清了Pandas中groupby操作的主要原理。下面来讲讲groupby之后的常见操作。

In [192]:
data.groupby(["company", "salary"])['age'].apply(list)

company  salary
A        13        [30]
         14        [39]
         21        [15]
B        7         [46]
         19        [45]
         29        [37]
         35        [20]
         40        [28]
C        10        [24]
         31        [38]
Name: age, dtype: object

进行了`groupby`之后，其实是形成了一个个的小dataframe：

In [193]:
data.groupby("company").apply(list)

company
A    [company, salary, age]
B    [company, salary, age]
C    [company, salary, age]
dtype: object

In [194]:
data.groupby("company")['company'].apply(list)

company
A          [A, A, A]
B    [B, B, B, B, B]
C             [C, C]
Name: company, dtype: object

如果你想单独操作某一个`list`，例如`salary`，你就在`groupby('company)`后面带上['salary'], 然后对每个公司的这个`salary`进行`apply function`数据操作

In [195]:
data.groupby("company")['salary'].apply(list)

company
A           [21, 14, 13]
B    [35, 29, 19, 7, 40]
C               [31, 10]
Name: salary, dtype: object

In [196]:
data.groupby("company")['salary'].apply(lambda x: x*2)

0    42
1    70
2    28
3    62
4    58
5    26
6    20
7    38
8    14
9    80
Name: salary, dtype: int64

最后的结果是成单独一列

# 操作1：agg简单聚合

最后的结果是一个聚合的结果

针对样例数据集，如果我想求不同公司员工的平均年龄和平均薪水，可以按照下方的代码进行：

In [197]:
data.groupby("company").agg('mean')

,salary,age
company,,
A,16.0,28.0
B,26.0,35.2
C,20.5,31.0


如果想对**针对不同的列求不同的值**，比如要计算不同公司员工的平均年龄以及薪水的中位数，可以利用字典进行聚合操作的指定：

In [198]:
data.groupby('company').agg({'salary':'median','age':'mean'})

,salary,age
company,,
A,14.0,28.0
B,29.0,35.2
C,20.5,31.0


`agg`聚合过程可以图解如下（第二个例子为例）：

![](https://pic3.zhimg.com/80/v2-c580eb0c4fec7d4b3de272f42bdb2fba_1440w.jpg)

# 操作2：`transform`

transform的作用为直接可以把聚合后的数据塞回原dataframe

如果想将每个公司的`avg_salary`塞入到原df，不使用transform需要两步，用了transform只需要一步：

无transform：


In [199]:
avg_salary_dict = data.groupby('company')['salary'].mean().to_dict()
data['avg_salary'] = data['company'].map(avg_salary_dict)
data

,company,salary,age,avg_salary
0,A,21,15,16.0
1,B,35,20,26.0
2,A,14,39,16.0
3,C,31,38,20.5
4,B,29,37,26.0
5,A,13,30,16.0
6,C,10,24,20.5
7,B,19,45,26.0
8,B,7,46,26.0
9,B,40,28,26.0


有transform：

In [200]:
data['avg_salary'] = data.groupby('company')['salary'].transform('mean')
data

,company,salary,age,avg_salary
0,A,21,15,16.0
1,B,35,20,26.0
2,A,14,39,16.0
3,C,31,38,20.5
4,B,29,37,26.0
5,A,13,30,16.0
6,C,10,24,20.5
7,B,19,45,26.0
8,B,7,46,26.0
9,B,40,28,26.0


![](https://pic1.zhimg.com/80/v2-47d83fb973be421545493e92dd0cf0d0_1440w.jpg)

图中的大方框是transform和agg所不一样的地方，对agg而言，会计算得到A，B，C公司对应的均值并直接返回，但对transform而言，则会对**每一条数据求得相应的结果，同一组内的样本会有相同的值**，组内求完均值后会**按照原索引的顺序**返回结果，如果有不理解的可以拿这张图和agg那张对比一下。

# 操作3：`apply`

区别是有的，但是整个实现原理是基本一致的。两者的区别在于，对于groupby后的apply，以分组后的`subDataFrame`作为参数传入指定函数的，基本操作单位是`DataFrame`，而之前介绍的apply的基本操作单位是Series。

apply灵活性巨大，因为`group-apply`的对象为**各个子dataframe**。

假设我现在需要获取各个公司年龄最大的员工的数据，该怎么实现呢？可以用以下代码实现：

In [201]:
company = ["A","B","C"]

data = pd.DataFrame({"company":[company[x] for x in np.random.randint(0,len(company),10)],
                   "salary":np.random.randint(5,50,10),
                   "age":np.random.randint(15,50,10)})

In [202]:
data

,company,salary,age
0,A,20,41
1,C,46,15
2,C,21,20
3,A,40,36
4,C,33,42
5,B,14,45
6,A,12,23
7,B,48,15
8,C,15,40
9,B,33,19


In [203]:
# 直接就是以df为输入

def get_oldest_staff(x):
  df = x.sort_values(by = 'age',ascending=True)
  return df.iloc[-1,:]

In [204]:
oldest_staff = data.groupby('company',as_index=False).apply(get_oldest_staff)

In [205]:
oldest_staff

,company,salary,age
0,A,20,41
1,B,14,45
2,C,33,42


你也可以直接得到dataframe拍扁了的结果：

In [206]:
data.groupby('company')['age'].apply(list)

company
A        [41, 36, 23]
B        [45, 15, 19]
C    [15, 20, 42, 40]
Name: age, dtype: object

当然你也可以直接apply+lambda，对所有subdf取最大值

In [207]:
data.groupby('company').apply(lambda x: x.max())

,company,salary,age
company,,,
A,A,40,41
B,B,48,45
C,C,46,42


或者只是对subdf里面的`salary`取最大值

In [208]:
data.groupby('company')[['salary']].apply(lambda x: x.max())

,salary
company,
A,40
B,48
C,46


In [209]:
data.groupby('company')[['salary']].max()

,salary
company,
A,40
B,48
C,46


![](https://pic4.zhimg.com/80/v2-aee9b9bda947364719876e6ffb861813_1440w.jpg)

可以看到，此处的apply和上篇文章中所介绍的作用原理基本一致，只是传入函数的参数由Series变为了此处的分组DataFrame。

最后，关于apply的使用，这里有个小建议，虽然说apply拥有更大的灵活性，但apply的运行效率会比agg和transform更慢。所以，groupby之后能用agg和transform解决的问题还是优先使用这两个方法，实在解决不了了才考虑使用apply进行操作。